### Evaluating the usage of different ML models for applicability

##### The first part of this was selecting the restaurants based on the reviews, or overviews, the second part (here), will be testing some ML models on the numerical data we have, like user ratings, number of reviews, maybe location too? Could add in something that computes location from user and added a penalty to restaurants further than 45 mins away or something

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from skorch import NeuralNetRegressor
import pickle
import emoji
import requests
from bs4 import BeautifulSoup


In [2]:
# loading merged dataset
initial_df = pd.read_csv('Data/Merged_TA_Gmaps_Dataset_2.0.csv')


In [3]:
df = initial_df.drop(['Unnamed: 0'],axis=1)

df = df.dropna(axis=0, how='any', subset = ['rating_review','rating','price_level','user_ratings_total'])

In [13]:
df_summary = df.groupby('restaurant_name').agg({'rating_review': 'mean', 'rating': 'mean', 'user_ratings_total': 'mean', 'id' : 'count'})
df_summary

,rating_review,rating,user_ratings_total,id
restaurant_name,,,,
108_Brasserie,4.395349,4.2,767.0,1118
12th_Knot,4.031977,4.4,1218.0,688
222_Vegan_Cuisine,4.276060,4.5,1465.0,873
2_Veneti,4.452471,4.4,482.0,263
34_Mayfair,4.190409,4.2,1333.0,709
...,...,...,...,...
Zizzi_Paddington_Street,4.042857,3.9,820.0,280
Zizzi_The_Strand,3.974684,4.2,1871.0,632
Zoilo,4.490909,4.6,554.0,440


In [29]:
# Obtaining all restaurants which have less than 50 reviews in total. This is an abitrary threshold applied to ensure that only restaurants
# with a statistically relevant number of reviews are considered. Typically if a place has less than 50 reviews, people are hesitant to try.
# There is a limitation to this item's effectiveness as this will become less relevant due to the age of the original dataset, ie. a restaurant
# may have had less than 50 reviews when this Google Maps data was pulled, but has since surpassed this.
low_vol_reviews = df_summary.loc[df_summary['user_ratings_total'] < 50,]

df_filtered = df.loc[~df['restaurant_name'].isin(low_vol_reviews.index)]

Index(['Artisan_Grill', 'Butler_s_at_the_Chesterfield', 'The_Blue_Door_Bistro',
       'Yo_Sushi_Harvey_Nichols'],
      dtype='object', name='restaurant_name')

In [ ]:
reg = LinearRegression().fit(df['rating_review'], )